In [2]:
!pip install dgl -f https://data.dgl.ai/wheels/torch-2.4/repo.html
!pip install matgl

import matgl
matgl.set_backend("DGL")

In [3]:
from pymatgen.core import Lattice, Structure

model = matgl.load_model("MEGNet-MP-2018.6.1-Eform")

# This is the structure obtained from the Materials Project.
struct = Structure.from_spacegroup("Pm-3m", Lattice.cubic(4.1437), ["Cs", "Cl"], [[0, 0, 0], [0.5, 0.5, 0.5]])
eform = model.predict_structure(struct)
print(f"The predicted formation energy for CsCl is {float(eform.numpy()):.3f} eV/atom.")

In [4]:
#!pip install "numpy<2"
#!pip install dgl==2.2.0
#!pip install torch==2.3.0
#!pip install "torchdata<=0.8.0"
!pip install xgboost
!pip install catboost
!pip install matgl

In [5]:
import torch
from pymatgen.core import Lattice, Structure

struct = Structure.from_spacegroup("Pm-3m", Lattice.cubic(4.1437), ["Cs", "Cl"], [[0, 0, 0], [0.5, 0.5, 0.5]])
model = matgl.load_model("MEGNet-MP-2019.4.1-BandGap-mfi")

In [6]:
# Load the pre-trained M3GNet formation energy model
model = matgl.load_model("M3GNet-MP-2018.6.1-Eform")
eform = model.predict_structure(struct)
print(f"The predicted formation energy for CsCl is {float(eform):.3f} eV/atom.")

# Extract the structure features of a structure
feat_dict = model.model.predict_structure(struct, return_features=True)
# Print out structure-wise features, it should be the dimension of node_features * 2 from set2set layer
print(feat_dict["readout"].shape)



In [7]:
import pandas as pd
import numpy as np
import plotly as pt
import seaborn as sns
#!pip install pymatgen
!pip install mp_api
import requests
import json
import matplotlib.pyplot as plt
!pip install matminer

#Initialization

In [1884]:
extract_CIFs = False
cif_folder = "/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/"

remove_outliers = True
remove_outliers_manually = True
remove_outliers_after_descriptors = False

update_IDs = False  #request material IDs from databases
target_col = 'Log_rate'
cross_validation = True
dataset_from_google_drive = True
grid_search = False
do_PCA = False
do_PCA_on_embeddings_only = True
do_LASSO = True
finite_log_of_zero_rate = False
embedding_type = 'RDF' #   'None',  'SiteStatsFingerprint' , 'M3GNet', 'RDF', 'CGCNN', 'electronic_RDF'

In [1885]:

df = None
if(dataset_from_google_drive):
  df = pd.read_excel("/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/Perovskite dataset export.xlsx",sheet_name='Photocatalytic dataset')
else:
  df = pd.read_excel("dataset.xlsx")

In [1886]:
df

In [1887]:
df.info()

In [1888]:
df.describe()

In [1889]:
df.columns

In [1890]:
init_columns = ['Perovskite', 'Interlayer space composition',
       'Bandgap, eV', 'Materials Project ID', 'COD_ID', 'Springer_ID',
       'Z', 'a, A', 'b, A', 'c, A',
       'Symmetry group', 'd,A',
       'Number of octahedrons on a layer', 'Prep Method', 'CalcT(K)', 'Calc time (h)',
       'Nitrogen', 'Promotion method', 'Promoter', 'Promoter, w%',
       'Surface area, m2/g', 'CatW, g/L', 'Alcohol, %', 'Sac Agent 1',
       'Sac agent 2', 'Light type', 'Power, W', 'Wave length min, nm',
       'Cell material', 'Temperature, K', 'Rate, umol/(g*h)']

init_columns = init_columns + ['MP_CIF_modifier', 'COD_CIF_modifier', 'Springer_CIF_modifier',
       'Materials Project verification', 'COD verification',
       'Springer verification', 'General verification', 'MP_CIF_modified',
       'COD_CIF_modified', 'Springer_CIF_modified']
print(init_columns)

In [1891]:
df = df[init_columns]
df.info()

#Formatting


In [1892]:
subs_map = {
    "Ph": "C6H5",
    "Bn": "C7H7",
    #"Pr": "C3H7",
    "Bu": "C4H9",
    "Hx": "C6H13",
    "Me": "CH3",
    "Et": "C2H5",
    "Oc": "C8H17",
    "Dc": "C10H21",
}

import re

def expand_substituents(formula):
    if pd.isna(formula):
        return formula

    for abbr, full in subs_map.items():
        formula = re.sub(rf'{abbr}', full, formula)
    return formula

In [1893]:
print(df.shape[0])
df = df[~df['Perovskite'].str.contains("Nx", na=False)]
df = df[~df['Perovskite'].str.contains("Ox", na=False)]
print(df.shape[0])

In [1894]:
df['Perovskite'] = df['Perovskite'].apply(expand_substituents)
#print(expand_substituents('HNdTiO4*MeNH2'))

In [1895]:
df = df.replace("-", np.nan)
df["a, A"] = df["a, A"].astype("float64")
df["b, A"] = df["b, A"].astype("float64")
df["c, A"] = df["c, A"].astype("float64")
df["Number of octahedrons on a layer"] = df["Number of octahedrons on a layer"].astype("float64")

In [1896]:
df["Nitrogen"] = df["Nitrogen"].fillna(False)
df["Nitrogen"]
df["Nitrogen"].astype("bool")
df["Nitrogen"].value_counts()

In [1897]:
df["Promoter, w%"] = df["Promoter, w%"].fillna(0)
df["Promoter, w%"].value_counts()

In [1898]:
df["Surface area, m2/g"] = df["Surface area, m2/g"].astype("float64")
df["Surface area, m2/g"].value_counts()

#df['Materials Project ID'] = df['Materials Project ID'].astype(str)
print(df['Materials Project ID'].dtype)

In [1899]:
df["Perovskite"].value_counts()

In [1900]:
df.info()

In [1901]:
df.to_excel("checkpoint_formatting.xlsx")


#Data cleaning

##Missing data handling

In [1902]:
df.isna().sum()

In [1903]:
df.columns

In [1904]:
drop_columns = ['Perovskite', 'Bandgap, eV',
       'Prep Method', 'CalcT(K)', 'Calc time (h)',
       'CatW, g/L', 'Light type',
       'Power, W',
       'Rate, umol/(g*h)']

drop_columns=['Perovskite','Rate, umol/(g*h)'] #drop nan in these columns

df['Bandgap, eV'] = df['Bandgap, eV'].fillna(df['Bandgap, eV'].mean())
df['Prep Method'] = df['Prep Method'].fillna(df['Prep Method'].mode()[0])
df['CalcT(K)'] = df['CalcT(K)'].fillna(df['CalcT(K)'].mean())
df['Calc time (h)'] = df['Calc time (h)'].fillna(df['Calc time (h)'].mean())
df['CatW, g/L'] = df['CatW, g/L'].fillna(df['CatW, g/L'].mean())
df['Light type'] = df['Light type'].fillna(df['Light type'].mode()[0])
df['Power, W'] = df['Power, W'].fillna(df['Power, W'].mean())

df.dropna(subset=drop_columns, inplace=True)
df.shape[0]

In [1905]:
df = df[df["General verification"] == False]
df.shape[0]

In [1906]:
df.isna().sum()

In [1907]:
def calculate_average_specific_area(dataframe):
  total=0;
  n=0;
  for index, row in df.iterrows():
    area =  row['Surface area, m2/g']
    if(np.isnan(area) or area > 55):
      continue
    total += area
    n+=1
  return total/n

In [1908]:
avr_specific_area = calculate_average_specific_area(df)
print(avr_specific_area)
df["Surface area, m2/g"] = df["Surface area, m2/g"].fillna(avr_specific_area)

df["Alcohol, %"] = df["Alcohol, %"].fillna(0)
df["Wave length min, nm"] = df["Wave length min, nm"].fillna(df["Wave length min, nm"].min())
df["Cell material"] = df["Cell material"].fillna(df["Cell material"].mode()[0])
print(df["Cell material"].mode()[0])
df["Temperature, K"] = df["Temperature, K"].fillna(298.15)
df["Materials Project ID"] = df["Materials Project ID"].fillna(-1)
df["COD_ID"] = df["COD_ID"].fillna(-1)
df["Springer_ID"] = df["Springer_ID"].fillna(-1)

In [1909]:
df["Promoter"] = df["Promoter"].fillna("No promoter")
df["Promotion method"] = df["Promotion method"].fillna("No promotion")

In [1910]:
if(finite_log_of_zero_rate):
  df['Rate, umol/(g*h)'].replace(0, 0.1, inplace=True)

df['Log_rate'] = np.log(df['Rate, umol/(g*h)'])
df['Log_rate'].replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna(subset=["Log_rate"])
df['Log_rate'].value_counts()

In [1911]:
#raise SystemExit("Stopping execution")
print(df.shape[0])
df = df[df["Sac Agent 1"].isna()]
df = df[df["Sac agent 2"].isna()]
print(df.shape[0])

In [1912]:
df["d,A"] = df["d,A"].fillna(df["d,A"].mean())
print(df["d,A"].mean())


In [1913]:
df.isna().sum()

## Duplicates handling

In [1914]:
duplicates = df[ df.duplicated()]
duplicates

##Handling outliers

In [1915]:
columns_outliers_detedection = [ 'Bandgap, eV',
      'Calc time (h)', 'Promoter, w%', 'Surface area, m2/g',
       'CatW, g/L', 'Alcohol, %',
       'Power, W', 'Wave length min, nm', 'Temperature, K']

columns_outliers_detedection = [ 'Bandgap, eV',
       'Promoter, w%', 'Surface area, m2/g',
       'CatW, g/L', 'Alcohol, %',
       'Power, W', 'Wave length min, nm']
#columns_outliers_detedection=[]

In [1916]:
import plotly.express as px
def plot_destribution(df,column):
  #fig = px.histogram(df, x=column,nbins=40,width=800, height=600, title=column)
  fig = px.violin(df, x=column,width=600, height=600, title=column)
  fig.update_layout(font_size=20)
  fig.show()
def plot_histogram(df,column):
  fig = px.histogram(df, x=column,nbins=20,width=600, height=600, title=column)
  fig.update_layout(font_size=20)
  fig.show()

In [1917]:
for column in columns_outliers_detedection:
  plot_histogram(df,column)

In [1918]:
min_boundries = {
    'Bandgap, eV':2,
    'Calc time (h)':-1,
    'Promoter, w%':-1,
    'Surface area, m2/g':-1,
    'CatW, g/L':-1,
    'Alcohol, %':-1,
    'Power, W':-1,
    'Wave length min, nm':-1,
    'Temperature, K':-1,
}
max_boundries = {
   'Bandgap, eV':4.5,
    'Calc time (h)':100,
    'Promoter, w%':6,
    'Surface area, m2/g':120,
    'CatW, g/L':6,
    'Alcohol, %':40,
    'Power, W':600,
    'Wave length min, nm':450,
    'Temperature, K':320,
}

In [1919]:
def manual_column_outliers_removal(_df, _column):
  num_rows_before = _df.shape[0]

  lower_bound = min_boundries.get(_column)
  upper_bound = max_boundries.get(_column)

  if(lower_bound is None):
    return _df

  df_clean = _df[(_df[_column]>= lower_bound) & (_df[_column] <= upper_bound)]
  num_rows_after = df_clean.shape[0]
  removed = num_rows_before - num_rows_after
  print(f'Column: {_column} removed ouliers: {removed }')
  print(f'min: {lower_bound} max: {upper_bound}')
  return df_clean

In [1920]:
def IQR_column_outliers_removal(_df, _column):
  num_rows_before = _df.shape[0]
  Q1 = _df[_column].quantile(0.25)
  Q3 = _df[_column].quantile(0.75)
  IQR = Q3 - Q1

  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  df_clean = _df[(_df[_column]>= lower_bound) & (_df[_column] <= upper_bound)]
  num_rows_after = df_clean.shape[0]
  removed = num_rows_before - num_rows_after
  print(f'Column: {_column} removed ouliers: {removed }')
  print(f'min: {lower_bound} max: {upper_bound}')
  return df_clean

In [1921]:
if not remove_outliers_after_descriptors and remove_outliers:
  for column in columns_outliers_detedection:
    if(remove_outliers_manually):
      df = manual_column_outliers_removal(df,column)
    else:
      df = IQR_column_outliers_removal(df,column)
print(f'Rows left: {df.shape[0]}')

#Featurizing





## Material Project IDs


In [1922]:
from mp_api.client import MPRester

API_KEY = "GFsoU5OV3dEngGT860TOtWcn35bE4y6l"
with MPRester(API_KEY) as mpr:
    materials = mpr.materials.search(formula="KCa2Nb3O10")
for material in materials:
    print(material.material_id, material.formula_pretty)

In [1923]:
def get_material_id_by_formula(f):
  with MPRester(API_KEY) as mpr:
    print(f"Composition: {f}")
    try:
      materials = mpr.materials.search(formula=f)
      if len(materials)>0:
        print(f"Composition: {f} ID is {materials[0].material_id}")
        return str(materials[0].material_id)
      else:
        print(f"Composition: {f} ID is not found")
        return str(-1)
    except Exception as e:
      print(f"Composition: {f} ID is not found")
      return str(-1)

In [1924]:
def update_Materials_Project_IDs(_df):
  for index, row in _df.iterrows():
        if pd.isna(row['Materials Project ID']) or row['Materials Project ID']==-1 or row['Materials Project ID']==-2 :
            new_id = get_material_id_by_formula(row['Perovskite'])
            print(f"New ID fetched: {new_id}  for formula {row['Perovskite']}")
            _df.at[index, 'Materials Project ID'] = str(new_id)
  return _df

In [1925]:
if update_IDs:
  df = update_Materials_Project_IDs(df)

##HIll's formula

In [1926]:
import re
from collections import Counter
def parse_formula(formula):
    """Extracts element counts from a chemical formula."""
    pattern = r"([A-Z][a-z]*)(\d*)"
    matches = re.findall(pattern, formula)

    element_counts = Counter()
    for element, count in matches:
        element_counts[element] += int(count) if count else 1  # Default to 1 if count is missing

    return element_counts
def to_hill_notation(formula):
    """Converts a chemical formula to Hill notation."""
    element_counts = parse_formula(formula)

    # Hill notation rules
    if "C" in element_counts:
        elements_sorted = ["C", "H"] if "H" in element_counts else ["C"]
        remaining_elements = sorted(e for e in element_counts if e not in ["C", "H"])
        elements_sorted.extend(remaining_elements)
    else:
        elements_sorted = sorted(element_counts.keys())
    hill_formula = " ".join(f"{el}{element_counts[el] if element_counts[el] > 1 else ''}" for el in elements_sorted)
    #print(f':{hill_formula}:')
    return hill_formula


In [1927]:
formulas = ["H2O", "C6H12O6", "NH3", "Fe2O3", "CH4", "CCl4", "H4C", "KCa2Nb3O10"]
for f in formulas:
    print(f"{f} -> {to_hill_notation(f)}")

In [1928]:
df['Hill formula']=df['Perovskite'].apply(to_hill_notation)
df

##COD_IDs


In [1929]:
def get_COD_ID_for_formula(formula):
  print(f"Formula :{formula}:")
  url=f"https://www.crystallography.net/cod/result?formula={formula}&format=json"
  response = requests.get(url)
  if response.status_code == 200:
    data = response.json()
    #print("Len: ",len(data))
    if(len(data)>0):
      #print(data[0]['file'])
      return data[0]['file']
  print(-1)
  return -1

def update_COD_ID(_df):
  for index, row in _df.iterrows():
        if pd.isna(row['COD_ID']) or row['COD_ID']==-1 or row['COD_ID']==-2 :
            print(f"New ID fetching start for formula {row['Hill formula']}")
            new_id = get_COD_ID_for_formula(row['Hill formula'])
            print(f"New ID fetched: {new_id}  for formula {row['Hill formula']}")
            _df.at[index, 'COD_ID'] = new_id
  return _df


In [1930]:
df.to_excel('checkpoint_Hill_formula.xlsx')
#print(get_COD_ID("O2 Ti"))
if update_IDs:
  df = update_COD_ID(df)

##CIF files extraction

In [1931]:
def extract_CIF_from_MP(material_id):
  if pd.isna(material_id) or material_id == "-1" or material_id == -1 or material_id == -2:
     print("Skipping invalid ID:", material_id)
     return
  print("ID: ", material_id)

  path = f'/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{material_id}.cif'
  if os.path.exists(path):
    print(f'CIF file for {material_id} already exist')
    return

  with MPRester(API_KEY) as mpr:
    #data = mpr.materials.search(material_ids=material_id)
    structure = mpr.materials.get_structure_by_material_id(material_id )
    cif_string = structure.to("struct.cif")

  ##with open(f"{material_id}.cif", "w") as f:
  #    f.write(cif_string)
  with open(f'/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{material_id}.cif', 'w') as f:
      f.write(cif_string)
  print("ID: ", material_id, " done!")

In [1932]:
import os
def extract_cif_from_COD(COD_ID):
  if(COD_ID==-1 or COD_ID==-2 ):
    return
  print(COD_ID)
  path = f'/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{COD_ID}.cif'
  if os.path.exists(path):
    print(f'CIF file for {COD_ID} already exist')
    return

  url = f"https://www.crystallography.net/cod/{COD_ID}.cif"
  response = requests.get(url)

  ##with open(f"{material_id}.cif", "w") as f:
  #    f.write(cif_string)
  if response.status_code == 200:
    print("Sucess")
    with open(path, 'w') as f:
        f.write(response.text)
  else:
    print('No CIF')

In [1933]:
if extract_CIFs:
  for material_id in df['COD_ID'].unique():
    extract_cif_from_COD(material_id)

In [1934]:
if extract_CIFs:
  for material_id in df['Materials Project ID'].unique():
    extract_CIF_from_MP(material_id)

##Valence electron discriptors

In [1935]:
from matminer.featurizers.composition import ElementProperty
from matminer.featurizers.composition import ValenceOrbital
from pymatgen.core.composition import Composition


In [1936]:
df['composition_obj'] = df['Hill formula'].apply(Composition)
print("Null compositions",df['composition_obj'].isna().sum())
vo_feat = ValenceOrbital()
df = vo_feat.featurize_dataframe(df, col_id='composition_obj')
df = df.drop('composition_obj', axis=1)

##Mulliken electronegativity

In [1937]:
from pymatgen.core.periodic_table import Element

def get_mulliken_en(element_symbol):
    el = Element(element_symbol)
    IE = el.ionization_energies[0]  # First ionization energy in eV
    EA = el.electron_affinity       # Electron affinity in eV

    if IE is None or EA is None:
        return None

    return (IE + EA) / 2


def calc_average_electronegativity(formula):
  # Example: For Fe2O3
  comp = Composition(formula)
  # Weighted mean electronegativity
  total_atoms = comp.num_atoms
  mean_en = sum(
    #comp[el] / total_atoms * Element(el).X
    comp[el] / total_atoms * get_mulliken_en(el)
    for el in comp.elements)
  return mean_en

In [1938]:
df["Average Mulliken electronegativity"] = df["Hill formula"].apply(calc_average_electronegativity)

##Valence electron density

In [1939]:
def split_formula(formula):
  output={}
  elements_with_indexes = formula.split()
  for el in elements_with_indexes:
    #match = re.match(r"([A-Za-z]+)(\d+)$", el)
    match = re.match(r"([A-Za-z]+)(\d+(?:\.\d+)?)$", el)
    if match:
      output[match.group(1)]=float(match.group(2))
    else:
      output[ el]=float(1)
  return output

def get_valence_electrons_number(hill_fomula):
  split = split_formula(hill_fomula)
  print(split)
  v = split.get("O")
  if v == None:
    return 0
  else:
    return 2*v

In [1940]:
df["Valence electrons"] = df["Hill formula"].apply(get_valence_electrons_number)

##CIF files descriptors

In [1941]:
from pymatgen.core.structure import Structure

def getCellVolume(ID):
  print("getCellVolume() ID:",ID)
  if(ID==-1):
    return 0
  file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{ID}.cif"
  if os.path.exists(file_path):
    try:
      structure = Structure.from_file(file_path)
    except:
      print('ERROR: Invalid structure for ',ID)
      return 0
  else:
    return 0

  if(structure == None):
    return 0
  return structure.volume

In [1942]:
def getCellZValue(ID):
  if(ID==-1):
    return 0
  file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{ID}.cif"
  if os.path.exists(file_path):
    try:
      structure = Structure.from_file(file_path)
    except:
      print('ERROR: Invalid structure for ',ID)
      return 0
  else:
    return 0
  if(structure == None):
    return 0
  return structure.composition.num_atoms / structure.composition.reduced_composition.num_atoms

In [1943]:
def abcExtractionFromMP(ID):
  if(ID==-1):
    return 0
  file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{ID}.cif"
  if os.path.exists(file_path):
    try:
      structure = Structure.from_file(file_path)
    except:
      print('ERROR: Invalid structure for ',ID)
      return [0,0,0]
  else:
    return  [0,0,0]

  if(structure == None):
    return  [0,0,0]
  a = structure.lattice.a
  b = structure.lattice.b
  c = structure.lattice.c
  return [a,b,c]

In [1944]:
CIF_source = "MP_CIF_modified"
if(extract_CIFs):
  CIF_source = "Materials Project ID"

df["Volume_MP"]=df[CIF_source].apply(getCellVolume)
df["Z_MP"]=df[CIF_source].apply(getCellZValue)
df['Valence Electrons Density_MP'] = (df['Valence electrons'] * df['Z_MP']) / df['Volume_MP']
df[['a_MP', 'b_MP', 'c_MP']] = df[CIF_source].apply(abcExtractionFromMP).apply(pd.Series)

In [1945]:
CIF_source = "COD_CIF_modified"
if(extract_CIFs):
  CIF_source = "COD_ID"

df["Volume_COD"]=df[CIF_source].apply(getCellVolume)
df["Z_COD"]=df[CIF_source].apply(getCellZValue)
df['Valence Electrons Density_COD'] = (df['Valence electrons'] * df['Z_COD']) / df['Volume_COD']
df[['a_COD', 'b_COD', 'c_COD']] = df[CIF_source].apply(abcExtractionFromMP).apply(pd.Series)

In [1946]:
CIF_source = "Springer_CIF_modified"
if(extract_CIFs):
  CIF_source = "Springer_ID"

df["Volume_Springer"]=df[CIF_source].apply(getCellVolume)
df["Z_Springer"]=df[CIF_source].apply(getCellZValue)
df['Valence Electrons Density_Springer'] = (df['Valence electrons'] * df['Z_Springer']) / df['Volume_Springer']
df[['a_Springer', 'b_Springer', 'c_Springer']] = df[CIF_source].apply(abcExtractionFromMP).apply(pd.Series)

In [1947]:
df["Volume_manual"]=df["a, A"]*df["b, A"]*df["c, A"]
df['Valence Electrons Density_manual'] = (df['Valence electrons'] * df['Z']) / df['Volume_manual']

##Oxygen concentration

In [1948]:
def count_oxigen(formula):
  #match = re.search(r'O(\d+)', hill_formula)
  #if match:
  #  number = int(match.group(1))
  #  print(number)
  #  return number
  #else:
  #  print("No match found")
  if formula is None:
    return 0
  print(formula)
  comp = Composition(formula)

  # Get number of oxygen atoms
  oxygen_count = comp.get_el_amt_dict().get("O", 0)
  print(oxygen_count)
  return oxygen_count

In [1949]:
df['Oxygen_count']=df['Hill formula'].apply(count_oxigen)

In [1950]:
df['Oxygen_concentration_manual']=df['Z']*df['Oxygen_count']/df['Volume_manual']
df['Oxygen_concentration_MP']=df['Z_MP']*df['Oxygen_count']/df['Volume_MP']
df['Oxygen_concentration_COD']=df['Z_COD']*df['Oxygen_count']/df['Volume_COD']
df['Oxygen_concentration_Springer']=df['Z_Springer']*df['Oxygen_count']/df['Volume_Springer']

##Packing fraction


In [1951]:
ionic_radii={"H+":0.02}
difficult_compunds_oxidation_states={
    "Ca Cs Na O10 Ta3":{"Ca":2,"Cs":1,"Na":1,"O":-2,"Ta":5},
    "Fe K2 La2 O10 Ti2":{"Fe":3, "K": 1,"La":3,"O":-2,"Ti":4},
    'Ca2 H Nb O10 Ta':{"Ca":2,"H":1,"Nb": 5,"Ta":5,"O":-2},
    'K2 O10 Sr Ta3':{"K":1,"O":-2,"Sr":2,"Ta":5},
    'H Nb O10 Sr2 Ta':{"H":1,"Nb":5,"O":-2,"Sr":2,"Ta":5},
}

In [1952]:
from pymatgen.core import Structure
from pymatgen.analysis.local_env import ValenceIonicRadiusEvaluator
def get_packing_fraction_from_cif(mp_id):
    print(mp_id)
    cif_folder = "/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/"
    cif_path = os.path.join(cif_folder, f"{mp_id}.cif")

    if not os.path.isfile(cif_path):
        return np.nan  # Return NaN if file doesn't exist
    try:
      structure = Structure.from_file(cif_path)
    except:
      return 0
    comp = structure.composition
    #comp_oxi = comp.oxidation_state_guesses()
    #print(comp_oxi)


    # Extract radius for each site
    site_radii = []

    try:
      print('Calculating oxidation states')
      structure.add_oxidation_state_by_guess()
    except:
      print('Error')
      return


    for site in structure:
        #el = Element(site.species_string)
        try:
          specie = site.specie
        except AttributeError:
          return 0
        print(site.specie.ionic_radii)
        print(site.specie.oxi_state)
        print('Specie ionic radii ',site.specie.ionic_radii)
        print('Site oxi satte ',site.specie.oxi_state)
        rad =site.specie.ionic_radii.get(site.specie.oxi_state)
        print('got site')
        if(rad is not None):
          site_radii.append(rad)

    # Compute atomic volume per site
    atom_volumes = [(4/3) * np.pi * r**3 for r in site_radii]
    total_atom_volume = sum(atom_volumes)

    # Compute packing fraction
    packing_fraction = total_atom_volume / structure.volume
    print('OK')
    return packing_fraction
    #except:
        #return np.nan  # Return NaN if anything fails (e.g., missing radii)

In [1953]:
from pymatgen.core import Structure
from pymatgen.analysis.local_env import ValenceIonicRadiusEvaluator
from pymatgen.core.periodic_table import Species

unresolved_compunds = []

def get_packing_fraction_from_formula_and_cell_volume(hill_formula, V, Z):
    if V==0 or np.isnan(V) or Z==0 or np.isnan(Z):
        return np.nan
    print("Enter!")
    print(hill_formula)
    print(V)
    print(Z)
    comp = Composition(hill_formula)
    oxi_guesses = comp.oxi_state_guesses()
    if(len(oxi_guesses)==0):
      oxi_guesses = difficult_compunds_oxidation_states.get(hill_formula)
      if(oxi_guesses==None):
        unresolved_compunds.append(hill_formula)
        return np.nan
    else:
      oxi_guesses = oxi_guesses[0]

    V_ions_formula=0
    for el, amt in comp.items():
        symbol = el.symbol
        el_oxidation_state = oxi_guesses[symbol]
        #element = Element(el)
        print("Element:", symbol)
        print("Element ox state:", el_oxidation_state)
        specie = Species(symbol,el_oxidation_state)
        r = specie.ionic_radius
        if(r==np.nan or r==None):
          ion_formula = symbol
          if(el_oxidation_state!=0 and el_oxidation_state!=1 and el_oxidation_state!=-1):
             ion_formula =  ion_formula+str(el_oxidation_state)
          if(el_oxidation_state>0):
            ion_formula =  ion_formula+str("+")
          if(el_oxidation_state<0):
            ion_formula =  ion_formula+str("-")
          print("Local ionic radii table request for ",ion_formula)
          r= ionic_radii.get(ion_formula)
          if(r==None):
            r=0
        print("r: ", r)
        V_ion = (4/3) * np.pi * r**3
        V_ions_formula += (V_ion*amt)
        print(el," ",amt)

    packing_fraction= Z*V_ions_formula/V
    return packing_fraction
    print("Output!")

In [1954]:
#fr = get_packing_fraction_from_formula_and_cell_volume("H2 La2 Ti3 O10",390.7,2)
#print("Fraction: ", fr)

#fr = get_packing_fraction_from_formula_and_cell_volume("O2 Ti",1183,30)
#print("Fraction: ", fr)

fr = get_packing_fraction_from_formula_and_cell_volume("Ca Cs Na O10 Ta3",228,1)
print("Fraction: ", fr)

In [1955]:
#df["MP_packing_fraction"] = df["Materials Project ID"].apply(get_packing_fraction_from_cif)
#df["COD_packing_fraction"] = df["COD_ID"].apply(get_packing_fraction_from_cif)
#df["Springer_packing_fraction"] = df["Springer_ID"].apply(get_packing_fraction_from_cif)

df["MP_packing_fraction"] = df.apply(lambda row: get_packing_fraction_from_formula_and_cell_volume(row["Hill formula"], row["Volume_MP"], row["Z_MP"]), axis=1)
df["COD_packing_fraction"] = df.apply(lambda row: get_packing_fraction_from_formula_and_cell_volume(row["Hill formula"], row["Volume_COD"], row["Z_COD"]), axis=1)
df["Springer_packing_fraction"] = df.apply(lambda row: get_packing_fraction_from_formula_and_cell_volume(row["Hill formula"], row["Volume_Springer"], row["Z_Springer"]), axis=1)
df["Manual_packing_fraction"] = df.apply(lambda row: get_packing_fraction_from_formula_and_cell_volume(row["Hill formula"], row["Volume_manual"], row["Z"]), axis=1)

In [1956]:
print(unresolved_compunds)

##Averaging


In [1957]:
def averaging_valence_electron_density(MP,COD,Springer):
  sum=0;
  count=0;
  print(MP)
  print(COD)
  print(Springer)
  if(pd.notna(MP)):
    sum+=MP
    count+=1
  if(pd.notna(COD)):
    sum+=COD
    count+=1
  if(pd.notna(Springer)):
    sum+=Springer
    count+=1
  print('Count: ',count)
  if count==0:
    return 0
  return sum/count


def averaging_4(MP,COD,Springer,manual):
  sum=0;
  count=0;
  print(MP)
  print(COD)
  print(Springer)
  print(manual)
  if(pd.notna(MP)):
    sum+=MP
    count+=1
  if(pd.notna(COD)):
    sum+=COD
    count+=1
  if(pd.notna(Springer)):
    sum+=Springer
    count+=1
  if(pd.notna(manual)):
    sum+=manual
    count+=1
  print('Count: ',count)
  if count==0:
    print("No value")
    return np.nan
  return sum/count

In [1958]:
df['Valence Electrons Density avg'] = df.apply(lambda x: averaging_4(x['Valence Electrons Density_MP'], x['Valence Electrons Density_COD'],x['Valence Electrons Density_Springer'],x['Valence Electrons Density_manual']), axis=1)
df['Oxygen_concentration avg'] = df.apply(lambda x: averaging_4(x['Oxygen_concentration_MP'], x['Oxygen_concentration_COD'],x['Oxygen_concentration_Springer'],x['Oxygen_concentration_manual']), axis=1)
df['Packing fraction avg'] = df.apply(lambda x: averaging_4(x['MP_packing_fraction'], x['COD_packing_fraction'],x['Springer_packing_fraction'],x['Manual_packing_fraction']), axis=1)

In [1959]:
df.to_excel('checkpoint_before_CIF_embeddings.xlsx')

##CIF embedings

In [1960]:
from pymatgen.core import Structure
from matminer.featurizers.site import CrystalNNFingerprint
from matminer.featurizers.structure import SiteStatsFingerprint
from pymatgen.transformations.standard_transformations import OrderDisorderedStructureTransformation
from matminer.featurizers.structure import JarvisCFID
from matminer.featurizers.structure import RadialDistributionFunction
from matminer.featurizers.structure.rdf import ElectronicRadialDistributionFunction


In [1961]:
matgl.get_available_pretrained_models()

['CHGNet-MPtrj-2023.12.1-2.7M-PES',
 'CHGNet-MPtrj-2024.2.13-11M-PES',
 'CHGNet-MatPES-PBE-2025.2.10-2.7M-PES',
 'CHGNet-MatPES-r2SCAN-2025.2.10-2.7M-PES',
 'M3GNet-ANI-1x-Subset-PES',
 'M3GNet-MP-2018.6.1-Eform',
 'M3GNet-MP-2021.2.8-DIRECT-PES',
 'M3GNet-MP-2021.2.8-PES',
 'M3GNet-MatPES-PBE-v2025.1-PES',
 'M3GNet-MatPES-r2SCAN-v2025.1-PES',
 'M3GNet-r2SCAN-NaBH',
 'MEGNet-MP-2018.6.1-Eform',
 'MEGNet-MP-2019.4.1-BandGap-mfi',
 'QET-MatQ-PES',
 'SO3Net-ANI-1x-Subset-PES',
 'TensorNet-ANI-1x-Subset-PES',
 'TensorNet-MatPES-PBE-v2025.1-PES',
 'TensorNet-MatPES-r2SCAN-v2025.1-PES',
 'TensorNetDGL-MatPES-PBE-v2025.1-PES',
 'TensorNetDGL-MatPES-r2SCAN-v2025.1-PES']

In [1962]:
model = matgl.load_model("MEGNet-MP-2019.4.1-BandGap-mfi")
state_attr = torch.tensor([0], dtype=torch.long)
output = model.predict_structure(struct, state_attr=state_attr)
print(output)

tensor(4.8254)


/usr/local/lib/python3.12/dist-packages/matgl/utils/io.py:159: UserWarning:

Incompatible model version detected! The code will continue to load the model but it is recommended that you provide a path to an updated model, increment your @model_version in model.json if you are confident that the changes are not problematic, or clear your ~/.matgl cache using `python -c "import matgl; matgl.clear_cache()"`



In [1963]:
from matgl.ext._pymatgen_dgl import Structure2Graph

converter = Structure2Graph(
    element_types=['Cs','Cl'],   # or explicit list like ["H", "C", "O", ...]
    cutoff=4.0            # must match model cutoff!
)

#g, lattice, state_attr = converter.get_graph(struct)
#output = model.forward(g,state_attr=state_attr)

In [1964]:
model = matgl.load_model("M3GNet-MP-2018.6.1-Eform")
#model = matgl.load_model("MEGNet-MP-2019.4.1-BandGap-mfi")
eform = model.predict_structure(struct)
print(f"The predicted formation energy for CsCl is {float(eform):.3f} eV/atom.")

# Extract the structure features of a structure
feat_dict = model.model.predict_structure(struct, return_features=True)
# Print out structure-wise features, it should be the dimension of node_features * 2 from set2set layer
print(feat_dict["readout"].shape)


The predicted formation energy for CsCl is -2.220 eV/atom.
torch.Size([1, 128])


In [1965]:
embedding_length = {
    'M3GNet':128,
    'SiteStatsFingerprint':122,
    'RDF':200,
    'CGCNN':1000,
    'electronic_RDF':200}


model = matgl.load_model("M3GNet-MP-2018.6.1-Eform")
jarvis = JarvisCFID() #122
featurizer_rdf = RadialDistributionFunction() #200
#featurizer_CGCNNF = dc.feat.CGCNNFeaturizer()
featurizer_electronic_rdf = ElectronicRadialDistributionFunction() #401 integer values

site_fp = CrystalNNFingerprint.from_preset("ops")
structure_featurizer = SiteStatsFingerprint(site_featurizer=site_fp)  #122

def extract_embedings_from_CIF(cif_file):
  if(embedding_type=='None'):
    return None
  #cif_folder = "/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/"
  cif_path = os.path.join(cif_folder, f"{cif_file}.cif")
  structure = Structure.from_file(cif_path)

  embedding= [None] * embedding_length[embedding_type]

  #structure.add_oxidation_state_by_guess()
  if structure.is_ordered:
        #structure.add_oxidation_state_by_guess()  # improves CrystalNN
        #embedding = structure_featurizer.featurize(structure)
        #embedding = featurizer_rdf.featurize(structure)
        #embedding = featurizer_CGCNNF.featurize(structure)
        #embedding =  featurizer_electronic_rdf.featurize(structure)
        #print(embedding)
        if(embedding_type=='M3GNet'):
          embedding = model.model.predict_structure(structure, return_features=True)
          embedding = embedding["readout"].detach().cpu().flatten().tolist()
        if(embedding_type=='SiteStatsFingerprint'):
          embedding = structure_featurizer.featurize(structure)
        if(embedding_type=='RDF'):
          embedding = featurizer_rdf.featurize(structure)
        if(embedding_type=='CGCNN'):
          embedding = None# featurizer_CGCNNF.featurize(structure)
        if(embedding_type=='electronic_RDF'):
          embedding = featurizer_rdf.featurize(structure)
  else:
       # df.at[idx, "disordered_structure"] = True
       # df.at[idx, "embedding"] = None  # cannot featurize
       print(f"Skipped structure-based features for {cif_ID} (disordered)")

  #embedding = structure_featurizer.featurize(structure)

  #embedding = jarvis.featurize(structure)
  print(len(embedding))
  print('Final: ',embedding)
  return embedding

In [1966]:
extract_embedings_from_CIF("mp-1245098")
extract_embedings_from_CIF("M_MP98")
#raise SystemExit("Stopping execution")

200
Final:  [  0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.          68.27679287 384.90958879 159.47807246
 144.30172933  67.85836984  16.52307377   3.78674548  41.80768861
  48.24170116 101.25185904 113.38040011 121.01270063 127.36720607
 101.16756608 103.27753644  73.26020158  72.67919612  84.34124579
  92.93300366  80.06167138 105.59356383  70.54910349 100.53360283
  93.08037398  71.64768698  94.94745446  79.57968699 100.33505136
  79.80942584  91.89001698 103.82026498  95.13762042  93.03978532
  93.49217726  88.31963977  95.6106295   95.72347522  95.05959272
  90.30697503  87.79381784 107.53990675  81.28243462  83.89026378
  77.71156252  87.37046689  76.03024931  88.17792304  93.50873578
  84.82533479  85.13113045 101.44815535  96.27915388  85.30146582
  87.94843997  92.86970458  92.31503871  94.07890759  99.10693656
  81.08852322  91.4909

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , 13.97097401, 37.72530316,
       45.51368851,  0.        ,  0.        ,  8.65913718,  0.        ,
       29.41709955,  0.        , 25.29429758, 82.42702677, 21.98114179,
       23.13395069, 19.27872396,  0.        ,  0.        , 16.07190177,
        0.        , 14.35895952,  0.        ,  6.45297518, 50.57576159,
        0.        ,  2.77688994, 10.5910244 , 32.85652174, 38.64181869,
       10.39556535,  0.        , 50.87259082, 34.56415405, 31.2299068 ,
        3.75067058,  7.2128643 ,  0.        ,  1.67092003,  0.        ,
       12.42133129, 32.96023668,  5.78614389, 33.54013512, 13.50926495,
        7.83982714, 21.49636337, 40.40364195, 16.60532895, 18.39364192,
        8.91815107, 13.51868386,  0.        , 10.1926478 , 11.88

In [1967]:
if(embedding_type is not 'None'):
  df["embedding"] = [None] * len(df)

  for idx, row in df.iterrows():
      cif_ID = 0
      if(row['Materials Project verification']):
        cif_ID = row['MP_CIF_modified']
      elif(row['COD verification']):
        cif_ID = row['COD_CIF_modified']
      elif(row['Springer verification']):
        cif_ID = row['Springer_CIF_modified']
      else:
        raise SystemExit("Stopping execution")
      print('CIF: ', cif_ID)
      #cif_path = os.path.join(cif_folder, f"{cif_ID}.cif")
      #structure = Structure.from_file(cif_path)

      embedding= extract_embedings_from_CIF(cif_ID)
      print('Insert embedding: ', embedding)
      df.at[idx, "embedding"] = embedding

<>:1: SyntaxWarning:

"is not" with 'str' literal. Did you mean "!="?

<>:1: SyntaxWarning:

"is not" with 'str' literal. Did you mean "!="?

/tmp/ipython-input-1239243898.py:1: SyntaxWarning:

"is not" with 'str' literal. Did you mean "!="?



KeyError: 'Materials Project verification'

In [1968]:
df.to_excel('checkpoint_cif_embeddings.xlsx')

In [1969]:
#df = pd.read_excel('checkpoint_cif_embeddings.xlsx')

#raise SystemExit("Stopping execution")

In [1970]:
if(embedding_type is not 'None'):
  df = df[df["embedding"].notna()]
  emb_df = pd.DataFrame(df["embedding"].to_list(), index=df.index)
  emb_df.columns = [f"emb_{i}" for i in range(emb_df.shape[1])]
  df = pd.concat([df, emb_df], axis=1)
  df.drop(columns=["embedding"], inplace=True)


In [1971]:
#!pip uninstall ase -y
#!pip install ase==3.22.1
#!pip install m3gnet

In [1972]:
if(embedding_type is not 'None'):
  print(df.shape[0])
  df = df.dropna(subset=["emb_0"])
  print(df.shape[0])
  print(df.isna().sum().sum())

In [1973]:
df.to_excel('checkpoint_cif_embeddings_split.xlsx')

In [1974]:
#from pymatgen.core import Structure
#from m3gnet.models import M3GNet

#test_cif = "/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/mp-557195.cif"
#structure = Structure.from_file(test_cif )
#model = M3GNet.load()

#embedding = model.get_structure_embedding(structure)
#print(embedding.shape)

#EDA

In [1975]:
df.info()

In [1976]:
df.columns

In [1977]:
columns_EDA = ['Perovskite','Bandgap, eV',
       'Symmetry group', 'd,A', 'Number of octahedrons on a layer',
       'Prep Method', 'CalcT(K)', 'Calc time (h)', 'Nitrogen',
       'Promotion method', 'Promoter', 'Promoter, w%', 'Surface area, m2/g',
       'CatW, g/L', 'Alcohol, %', 'Light type',
       'Power, W', 'Wave length min, nm', 'Cell material', 'Temperature, K',
       'Rate, umol/(g*h)',
       'avg s valence electrons', 'avg p valence electrons',
       'avg d valence electrons', 'avg f valence electrons',
       'frac s valence electrons', 'frac p valence electrons',
       'frac d valence electrons', 'frac f valence electrons',
       'Average Mulliken electronegativity', 'Valence electrons',
       'Valence Electrons Density avg', 'Oxygen_concentration avg',
       'Packing fraction avg','Log_rate']

In [1978]:
categorical_cols = [col for col in columns_EDA
                    if df[col].dtype in ['object', 'category','bool']]

numerical_cols = [col for col in columns_EDA
                  if df[col].dtype in ['int64', 'float64']]
print(categorical_cols)
print(numerical_cols)

In [1979]:
import plotly.express as px
from scipy.stats import gaussian_kde
import plotly.graph_objects as go

def plot_smooth_distribution(df, column):
    data = df[column].dropna().values

    kde = gaussian_kde(data)
    x_vals = np.linspace(data.min(), data.max(), 500)
    y_vals = kde(x_vals)

    # Plot
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=x_vals,
        y=y_vals,
        mode='lines',
        line=dict(width=3),
        name=f'{column} KDE'
    ))

    fig.update_layout(
        title=f"Distribution of '{column}'",
        xaxis_title=column,
        yaxis_title="Density",
        #template="plotly_white"
    )

    fig.show()

In [1980]:
plot_smooth_distribution(df, 'Bandgap, eV')

In [1981]:
for col in numerical_cols:
    fig = px.histogram(df, x=col, nbins=20, title=f'Histogram of {col}',width=800, height=500)
    fig.show()

In [1982]:
def draw_pie_chart(df, column):
  fig = px.pie(df, names=column, title=f'{column} Distribution',hole=0.3)
  fig.update_traces(textinfo='percent+label')
  fig.show()

In [1983]:
for col in categorical_cols:
     draw_pie_chart(df,col)

In [1984]:
from scipy.stats import pearsonr

def draw_scatter_plot(df,x_column, y_column):
  from sklearn.metrics import r2_score

  x = df[x_column]
  y = df[y_column]

  r, p_value = pearsonr(x, y)

  fig = go.Figure()

  fig.add_trace(go.Scatter(
      x=x,
      y=y,
      mode='markers',
      name='Data',
      marker=dict(size=8, color='blue', opacity=0.7)
  ))

  fig.add_annotation(
    x=np.mean(x),
    y=np.max(y),
    text=f"Pearson r = {r:.3f}",
    showarrow=False,
    font=dict(size=14, color="red")
  )

  fig.update_layout(
      xaxis_title=f'{x_column}',
      yaxis_title=f'{y_column}',
      template="plotly_white",
      width=600,
      height=600,
  )

  fig.show()

In [1985]:
for col in numerical_cols:
  draw_scatter_plot(df,col,'Rate, umol/(g*h)')

In [1986]:
for col in numerical_cols:
  draw_scatter_plot(df,col,target_col)

df_selected = df[numerical_cols]


In [1987]:
def draw_corr_heatmap(_df):
  # Compute correlation matrix
  corr_matrix = _df.corr()

  # Plot heatmap
  fig = px.imshow(
      corr_matrix,
      text_auto=True,        # show correlation values
      aspect="auto",
      color_continuous_scale='RdBu_r',  # diverging color map
      title="Correlation Heatmap",
      width=1000,
      height=1000,
  )
  fig.show()

In [1988]:
draw_corr_heatmap(df_selected)

In [1989]:
df.to_excel('checkpoint_descriptors.xlsx')

#Preparing data for ML

##Categorization


In [1990]:
#raise SystemExit("Stopping execution")

In [1991]:
categorical_columns = df.select_dtypes(include='object').columns.tolist()
categorical_columns.remove('Perovskite')
categorical_columns.remove('Interlayer space composition')
categorical_columns.remove('Materials Project ID')
categorical_columns.remove('COD_ID')
categorical_columns.remove('Springer_ID')
categorical_columns.remove('Symmetry group')
categorical_columns.remove('Sac Agent 1')
categorical_columns.remove('Sac agent 2')
categorical_columns.remove('Hill formula')

columns_to_remove = ['MP_CIF_modifier',
 'COD_CIF_modifier',
 'Springer_CIF_modifier',
 'MP_CIF_modified',
 'COD_CIF_modified',
 'Springer_CIF_modified']

for c in columns_to_remove:
  categorical_columns.remove(c)

categorical_columns

In [1992]:
df2 = df['Light type'].value_counts()
print(df2.index)

In [1993]:
df[categorical_columns].isna().sum()

In [1994]:
for col in df.columns:
    print(col)

In [1995]:
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)
df.columns


In [1996]:
df.info()

In [1997]:
columns_for_ML = df.columns
columns_to_exlude = ['Perovskite', 'Interlayer space composition','Materials Project ID', 'COD_ID', 'Springer_ID', 'Z', 'a, A', 'b, A',
       'c, A', 'Symmetry group', 'Number of octahedrons on a layer','Sac Agent 1',
       'Sac agent 2','Rate, umol/(g*h)','Hill formula','Volume_MP', 'Z_MP',
       'Valence Electrons Density_MP', 'a_MP', 'b_MP', 'c_MP', 'Volume_COD',
       'Z_COD', 'Valence Electrons Density_COD', 'a_COD', 'b_COD', 'c_COD',
       'Volume_Springer', 'Z_Springer', 'Valence Electrons Density_Springer',
       'a_Springer', 'b_Springer', 'c_Springer', 'Volume_manual',
       'Valence Electrons Density_manual', 'Oxygen_count',
       'Oxygen_concentration_manual', 'Oxygen_concentration_MP',
       'Oxygen_concentration_COD', 'Oxygen_concentration_Springer',
       'MP_packing_fraction', 'COD_packing_fraction',
       'Springer_packing_fraction', 'Manual_packing_fraction']
columns_to_exlude += ['MP_CIF_modifier',
'COD_CIF_modifier',
'Springer_CIF_modifier',
'Materials Project verification',
'COD verification',
'Springer verification',
'General verification',
'MP_CIF_modified',
'COD_CIF_modified',
'Springer_CIF_modified',]

columns_for_ML = [x for x in columns_for_ML if x not in columns_to_exlude]
print(columns_for_ML)


In [1998]:
df = df[columns_for_ML]
df.info()

In [1999]:
df["Valence Electrons Density avg"] = df["Valence Electrons Density avg"].fillna(df["Valence Electrons Density avg"].mean())
df["Oxygen_concentration avg"] = df["Oxygen_concentration avg"].fillna(df["Oxygen_concentration avg"].mean())
df["Packing fraction avg"] = df["Packing fraction avg"].fillna(df["Packing fraction avg"].mean())

In [2000]:

features_to_remove=['Oxygen_concentration avg','Valence electrons']
df = df.drop(columns=features_to_remove)

In [2001]:
print(df.isna().sum().sum())
if(df.isna().sum().sum() != 0):
  print(df.isna().sum())
  raise StopExecution()


In [2002]:
features = list(df.columns)
features.remove('Log_rate')

for f in features:
  print(f)

## Late outlier detection


In [2003]:
if(remove_outliers_after_descriptors and remove_outliers):
  #raise SystemExit("Stopping execution")
  from sklearn.preprocessing import RobustScaler
  print(df.shape[0])
  scaler = RobustScaler()

  df_scaled = df.copy()
  df_scaled[features] = scaler.fit_transform(df[features])
  print(df.shape[0])
  df = df_scaled
else:
  print('Did nothing')

In [2004]:
if(remove_outliers_after_descriptors and remove_outliers):
  #from sklearn.decomposition import PCA

  print(df.shape[0])

  X = df_scaled[features]

  #pca = PCA(n_components=0.95, random_state=0)
  #X_pca = pca.fit_transform(X)

  dist = np.linalg.norm(X, axis=1)


  threshold = np.percentile(dist, 80)
  print('Treshold = ',threshold)
  #threshold = X.std()
  #print('Treshold = ',threshold)
  df_clean = df[dist < threshold]

  print(df_clean.shape[0])
  df_clean.info()
  df = df_clean
else:
  print('Did nothing')

##Normalization

In [2005]:
df.info()

In [2006]:
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()

df_normalized = df.copy()
df_normalized[features] = scaler.fit_transform(df_normalized[features])
df_normalized.head()

In [2007]:
draw_corr_heatmap(df_normalized)

##Removing correlated features

In [2008]:
var = df_normalized.var()
df_normalized = df_normalized.loc[:, var > 1e-8]

print(df_normalized.info())

corr = df_normalized.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
to_drop = [c for c in upper.columns if any(upper[c] > 0.95)]
df_normalized = df_normalized.drop(columns=to_drop)
print('Mutual correlation')
print(df_normalized.info())

In [2009]:
draw_corr_heatmap(df_normalized)

#XY Split

In [2010]:
X = df_normalized.drop(columns=[target_col])
y = df_normalized[target_col]

In [2011]:
df_normalized.info()

In [2012]:
df_normalized.to_excel('df_normalized.xlsx')

#Features selection

In [2013]:
subset_cols = X.filter(like="emb_").columns.tolist()
print(subset_cols)

In [2014]:
from sklearn.decomposition import PCA

n_comp =30

#raise SystemExit("Stopping execution")

pca = PCA(n_components=n_comp, random_state=0)

if(do_PCA):
  columns_for_PCA = X.columns
  if(do_PCA_on_embeddings_only):
    columns_for_PCA = []
    columns_for_PCA = X.filter(like="emb_").columns.tolist()


  X_for_pca = X[columns_for_PCA]
  X_pca = pca.fit_transform(X_for_pca)
  explained = np.cumsum(pca.explained_variance_ratio_)
  plt.plot(explained, marker="o")
  plt.xlabel("Number of components")
  plt.ylabel("Cumulative explained variance")
  plt.grid()

  loadings = pd.DataFrame(
    pca.components_.T,
    columns=[f"PC{i+1}" for i in range(pca.n_components_)],
    index=X_for_pca.columns
  )
  X_for_pca.to_excel("checkpoint_X_before_PCA.xlsx")

  print(loadings)
  loadings.to_excel('PCA_loadings.xlsx')

  columns_after_PCA = [f'PC{i+1}' for i in range(pca.n_components_)]
  X_pca = pd.DataFrame(
      pca.fit_transform(X_for_pca),
      columns=columns_after_PCA,
      index=df.index
  )

  X_pca[columns_after_PCA] = scaler.fit_transform(X_pca[columns_after_PCA])

  df_pca = pd.concat([X_pca, y], axis=1)
  X = pd.concat([X, X_pca], axis=1)
  X = X.drop(columns_for_PCA, axis=1)
  df_pca.to_excel("checkpoint_ML_input.xlsx")
  X_pca.to_excel("checkpoint_X_after_PCA.xlsx")

#X_val_pca = pca.transform(scaler.transform(X_val))
#X_pca.info()
else:
  print('Do nothing')

In [2015]:
X.describe()

#ML


In [2016]:
#raise SystemExit("Stopping execution")


print('Std:',y.std())

print(X.shape)

In [2017]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold

In [2018]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True
)


In [2019]:
if(do_LASSO):
  from sklearn.linear_model import Lasso
  from sklearn.linear_model import LassoCV

  lasso_cv = LassoCV(cv=5, max_iter=10000)
  lasso_cv.fit(X_train, y_train)
  print("Optimal alpha:", lasso_cv.alpha_)

  # Use the selected alpha
  lasso_best = Lasso(alpha=lasso_cv.alpha_)
  lasso_best.fit(X_train, y_train)


  #lasso = Lasso(alpha=0.01, max_iter=10000)
  #lasso.fit(X_train, y_train)
  coef = pd.Series(lasso_best.coef_, index=X.columns)

  # Selected features (non-zero coefficients)
  selected_features = coef[coef != 0].index.tolist()
  print("Selected features:", selected_features)
  print("Values: ",coef[coef != 0])
  print(len(selected_features))
  X = X[selected_features]
  X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True
  )
  print(X.shape)
else:
  print("Do nothing")

In [2020]:
#y_pred = lasso.predict(X_test)
#print("Test R²:", r2_score(y_test, y_pred))

In [2021]:
def draw_actual_predicted(_y_test, _y_pred):
  fig = px.scatter(
    x=_y_test,
    y=_y_pred,
    labels={'x':'Actual', 'y':'Predicted'},
    title='Actual vs Predicted',
    width = 700,
    height = 700,
  )
  fig.add_shape(
      type="line",
      #x0=_y_test.min(), y0=_y_pred.min(),
      #x1=_y_test.max(), y1=_y_pred.max(),
      x0=-5, y0=-5,
      x1=12, y1=12,
      line=dict(color="red", dash="dash")
  )
  rmse = np.sqrt(mean_squared_error(_y_test, _y_pred))
  r2 = r2_score(_y_test, _y_pred)

  fig.add_annotation(
    x=0.05, y=0.95, xref="paper", yref="paper",
    text=f"R² = {r2:.4f}<br>RMSE = {rmse:.4f}",
    showarrow=False,
    font=dict(size=14)
  )
  fig.show()

##Linear regression

In [2022]:
linear_model = LinearRegression()

if(cross_validation):
  cv = KFold(n_splits=5, shuffle=True)
  scores = cross_val_score(linear_model, X, y, cv=cv, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(linear_model, X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  print('No cross validation')
  linear_model.fit(X_train, y_train)
  y_pred = linear_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

In [2023]:
def estimate_performance(_y_test, _y_pred):
  raisemse = np.sqrt(mean_squared_error(_y_test, _y_pred))
  r2 = r2_score(_y_test, _y_pred)

  print("RMSE:", mse)
  print("R² Score:", r2)

##Decision tree regression

In [2024]:
from sklearn.tree import DecisionTreeRegressor

In [2025]:
tree_model = DecisionTreeRegressor(random_state=42)

if(cross_validation):
  scores = cross_val_score(tree_model, X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(tree_model, X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  print('No cross validation')
  tree_model.fit(X_train, y_train)
  y_pred = tree_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

In [2026]:
import matplotlib.pyplot as plt

if not(cross_validation):
  feat_imp = pd.DataFrame({
      'Feature': X.columns,
      'Importance': tree_model.feature_importances_
  }).sort_values(by='Importance', ascending=False)

  plt.figure(figsize=(8,6))
  sns.barplot(x='Importance', y='Feature', data=feat_imp)
  plt.title('Feature Importance')
  plt.show()

## Random forest

In [2027]:
from sklearn.ensemble import RandomForestRegressor

In [2028]:
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=5,
    min_samples_split = 10,
    min_samples_leaf = 1,
    max_features=2,
    random_state=42
)

if(cross_validation):
  scores = cross_val_score(rf_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(rf_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  print('No cross validation')
  rf_model.fit(X_train, y_train)
  y_pred = rf_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

In [2029]:
if not(cross_validation):
  feat_imp = pd.DataFrame({
      'Feature': X.columns,
      'Importance': rf_model.feature_importances_
  }).sort_values(by='Importance', ascending=False)

  plt.figure(figsize=(8,6))
  sns.barplot(x='Importance', y='Feature', data=feat_imp)
  plt.title('Feature Importance')
  plt.show()

In [2030]:
from sklearn.model_selection import GridSearchCV

if(grid_search):
  param_grid = {
      'n_estimators': [50, 100, 150],
      'max_depth': [4, 5],
      'min_samples_split': [5, 10],
      'min_samples_leaf': [1, 2],
  }
  #Best params {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
  #Best params {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
  #Best params {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 100}

  grid_search = GridSearchCV(RandomForestRegressor(random_state=42),
                            param_grid, cv=5, scoring='neg_mean_squared_error')
  grid_search.fit(X,y)
  print('Best params',grid_search.best_params_)
  print('Best score',grid_search.best_score_)

In [2031]:
if(grid_search):
  best_model = grid_search.best_estimator_

  if(cross_validation):
    scores = cross_val_score(best_model , X, y, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-scores)
    print("Scores: ", scores)
    print("Mean RMSE:", rmse_scores.mean())

    y_pred = cross_val_predict(best_model , X, y, cv=5)
    draw_actual_predicted(y, y_pred)
  else:
    y_pred = best_model.predict(X_test)
    draw_actual_predicted(y_test, y_pred)

##Gradient boosting

In [2032]:
from sklearn.ensemble import GradientBoostingRegressor

In [2033]:
gb_model = GradientBoostingRegressor(
    n_estimators=100,      # number of boosting stages
    learning_rate=0.1,     # shrinkage factor
    max_depth=4,           # depth of each tree
    min_samples_split=6,
    min_samples_leaf=2,
    random_state=42,
    subsample=0.8
)

if(cross_validation):
  scores = cross_val_score(gb_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(gb_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  gb_model.fit(X_train, y_train)
  y_pred = gb_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

In [2034]:
if not(cross_validation):
  feat_imp = pd.DataFrame({
      'Feature': X.columns,
      'Importance': gb_model.feature_importances_
  }).sort_values(by='Importance', ascending=False)

  plt.figure(figsize=(8,6))
  sns.barplot(x='Importance', y='Feature', data=feat_imp)
  plt.title('Feature Importance')
  plt.show()

In [2035]:
if(grid_search):
  param_grid = {
      'n_estimators': [100, 150],        # number of boosting stages
      'learning_rate': [0.01, 0.1],   # shrinkage factor
      'max_depth': [3, 4],                # depth of each tree
      'min_samples_split': [4, 6],
      'min_samples_leaf': [1, 2, 3],
      'subsample': [0.8, 1.0]                # fraction of samples for each tree
  }
  #Best parameters: {'learning_rate': 0.1, 'max_depth': 4, 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 100, 'subsample': 0.8}
  #Best parameters: {'learning_rate': 0.1, 'max_depth': 4, 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 100, 'subsample': 0.8}
  #Best parameters: {'learning_rate': 0.1, 'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 100, 'subsample': 0.8}

  # Grid search with 5-fold cross-validation
  grid_search = GridSearchCV(
      estimator=gb_model,
      param_grid=param_grid,
      cv=5,
      scoring='neg_mean_squared_error',          # or 'neg_mean_squared_error'
      n_jobs=-1,
      verbose=2
  )

  # Fit on training data
  grid_search.fit(X, y)

  # Best hyperparameters
  print("Best parameters:", grid_search.best_params_)
  print("Best CV RMSE:", grid_search.best_score_)

In [2036]:
if(grid_search):
  best_model = grid_search.best_estimator_

  if(cross_validation):
    scores = cross_val_score(best_model , X, y, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-scores)
    print("Scores: ", scores)
    print("Mean RMSE:", rmse_scores.mean())

    y_pred = cross_val_predict(best_model , X, y, cv=5)
    draw_actual_predicted(y, y_pred)
  else:
    y_pred = best_model.predict(X_test)
    draw_actual_predicted(y_test, y_pred)

##KNN

In [2037]:
from sklearn.neighbors import KNeighborsRegressor

In [2038]:
knn_model = KNeighborsRegressor(n_neighbors=20, weights='uniform',p=1)

if(cross_validation):
  scores = cross_val_score(knn_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(knn_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  knn_model.fit(X_train, y_train)
  y_pred = knn_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

In [2039]:
if(grid_search):
  param_grid = {
      'n_neighbors': list(range(1, 21)),
      'weights': ['uniform', 'distance'],
      'p': [1, 2]
  }
  #Best parameters: {'n_neighbors': 20, 'p': 1, 'weights': 'uniform'}
  #Best parameters: {'n_neighbors': 20, 'p': 1, 'weights': 'uniform'}
  #Best parameters: {'n_neighbors': 12, 'p': 2, 'weights': 'distance'}

  grid_search = GridSearchCV(
      KNeighborsRegressor(),
      param_grid,
      cv=5,
      scoring='neg_mean_squared_error',
      n_jobs=-1,
      verbose=2
  )

  grid_search.fit(X,y)

  print("Best parameters:", grid_search.best_params_)
  print("Best CV RMSE:", grid_search.best_score_)

In [2040]:
if(grid_search):
  best_model = grid_search.best_estimator_

  if(cross_validation):
    scores = cross_val_score(best_model , X, y, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-scores)
    print("Scores: ", scores)
    print("Mean RMSE:", rmse_scores.mean())

    y_pred = cross_val_predict(best_model , X, y, cv=5)
    draw_actual_predicted(y, y_pred)
  else:
    y_pred = best_model.predict(X_test)
    draw_actual_predicted(y_test, y_pred)

##Catboost

In [2041]:
from catboost import CatBoostRegressor

In [2042]:
cat_boost_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=4,
    verbose=100
)

if(cross_validation):
  scores = cross_val_score(cat_boost_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(cat_boost_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  cat_boost_model.fit(X_train, y_train)
  y_pred = cat_boost_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

In [2043]:
if(grid_search):
  param_grid = {
      'depth': [4,6],
      'learning_rate':[0.01,0.05,0.1 ]
  }
  #Best parameters: {'depth': 4, 'learning_rate': 0.05}
  #Best parameters: {'depth': 4, 'learning_rate': 0.05}
  #Best parameters: {'depth': 6, 'learning_rate': 0.05}

  grid_search = GridSearchCV(cat_boost_model, param_grid, cv=5,scoring='neg_mean_squared_error',)
  grid_search.fit(X,y)

  print("Best parameters:", grid_search.best_params_)
  print("Best CV RMSE:", grid_search.best_score_)

In [2044]:
if(grid_search):
  est_model = grid_search.best_estimator_

  if(cross_validation):
    print("Cross validation")
    scores = cross_val_score(best_model , X, y, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-scores)

    y_pred = cross_val_predict(best_model , X, y, cv=5)
    draw_actual_predicted(y, y_pred)
    print("Scores: ", scores)
    print("Mean RMSE:", rmse_scores.mean())
  else:
    y_pred = best_model.predict(X_test)
    draw_actual_predicted(y_test, y_pred)

##XGBoost

In [2045]:
import xgboost as xgb
from xgboost import XGBRegressor

In [2046]:
xgboost_model = XGBRegressor(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

In [2047]:
if(cross_validation):
  scores = cross_val_score(xgboost_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(xgboost_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  xgboost_model.fit(X_train, y_train)
  y_pred = xgboost_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

In [2048]:
X_Maliken = X[['Average Mulliken electronegativity']]

if(cross_validation):
  cv = KFold(n_splits=5, shuffle=True)
  scores = cross_val_score(linear_model, X_Maliken, y, cv=cv, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(linear_model, X_Maliken, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  print('No cross validation')
  linear_model.fit(X_train, y_train)
  y_pred = linear_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)